# CrossFit Retention Project — 01 Exploratory Analysis
Initial EDA on raw PushPress exports: Members, Attendance, Store Sales, and Cancellations.

**Goal**: Understand data quality, build basic retention metrics, and surface quick insights for the owner.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

members = pd.read_csv(r"/mnt/data/Members.csv", parse_dates=["Join_Date"])
attendance = pd.read_csv(r"/mnt/data/Attendance.csv", parse_dates=["Date"])
sales = pd.read_csv(r"/mnt/data/Store_Sales.csv", parse_dates=["Purchase_Date"])
cancellations = pd.read_csv(r"/mnt/data/Cancellations.csv", parse_dates=["Cancel_Date"])

members.head(3), attendance.head(3), sales.head(3), cancellations.head(3)

## Basic Shapes & Nulls

In [ ]:
for name, df in {
    "members": members,
    "attendance": attendance,
    "sales": sales,
    "cancellations": cancellations,
}.items():
    print(name, df.shape)
    display(df.isna().mean().sort_values(ascending=False).to_frame("null_rate"))

## Member Growth Over Time

In [ ]:
members_by_month = (
    members.assign(month=members["Join_Date"].dt.to_period("M").dt.to_timestamp())
    .groupby("month")
    .size()
)
plt.figure()
members_by_month.plot(kind="bar")
plt.title("New Members per Month")
plt.xlabel("Month")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

## Cancellations Over Time

In [ ]:
cancels_by_month = (
    cancellations.assign(month=cancellations["Cancel_Date"].dt.to_period("M").dt.to_timestamp())
    .groupby("month")
    .size()
)
plt.figure()
cancels_by_month.plot(kind="bar")
plt.title("Cancellations per Month")
plt.xlabel("Month")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

## Current Churn Snapshot

In [ ]:
active_ids = set(members["Member_ID"]) - set(cancellations["Member_ID"])
churn_rate = len(set(cancellations["Member_ID"])) / len(set(members["Member_ID"]))
print(
    f"Members: {members['Member_ID'].nunique()} | Cancelled: {cancellations['Member_ID'].nunique()} | Churn Rate: {churn_rate:.2%}"
)
print(f"Active Members (approx): {len(active_ids)}")

## Attendance Patterns

In [ ]:
# Visits per member
visits_per_member = attendance[attendance["Status"].eq("Checked In")].groupby("Member_ID").size()
plt.figure()
visits_per_member.hist(bins=30)
plt.title("Distribution of Visits per Member")
plt.xlabel("Visits")
plt.ylabel("Members")
plt.tight_layout()
plt.show()

# Peak days/times
att = attendance.copy()
att["weekday"] = att["Date"].dt.day_name()
by_weekday = (
    att[att["Status"].eq("Checked In")]
    .groupby("weekday")
    .size()
    .reindex(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
)
plt.figure()
by_weekday.plot(kind="bar")
plt.title("Check-ins by Weekday")
plt.xlabel("Weekday")
plt.ylabel("Check-ins")
plt.tight_layout()
plt.show()

## Membership Type Mix & Referral Sources

In [ ]:
plt.figure()
members["Membership_Type"].value_counts().plot(kind="bar")
plt.title("Membership Type Distribution")
plt.xlabel("Type")
plt.ylabel("Members")
plt.tight_layout()
plt.show()

plt.figure()
members["Referral_Source"].value_counts().plot(kind="bar")
plt.title("Referral Source Distribution")
plt.xlabel("Source")
plt.ylabel("Members")
plt.tight_layout()
plt.show()

## Retail Sales Overview

In [ ]:
sales["month"] = sales["Purchase_Date"].dt.to_period("M").dt.to_timestamp()
sales_monthly = sales.groupby("month")["Amount_USD"].sum()
plt.figure()
sales_monthly.plot(kind="bar")
plt.title("Retail Revenue by Month")
plt.xlabel("Month")
plt.ylabel("USD")
plt.tight_layout()
plt.show()

top_items = sales.groupby("Item")["Amount_USD"].sum().sort_values(ascending=False)
display(top_items.to_frame("Revenue_USD"))

## Quick Retention Signals (Heuristics)

In [ ]:
# Days since last check-in
last_seen = (
    attendance[attendance["Status"].eq("Checked In")]
    .groupby("Member_ID")["Date"]
    .max()
    .rename("Last_Checkin")
)
features = members.set_index("Member_ID").join(last_seen).reset_index()
snapshot_date = max(attendance["Date"])
features["Days_Since_Last"] = (snapshot_date - features["Last_Checkin"]).dt.days

# Visit frequency (visits per active month)
first_seen = (
    attendance[attendance["Status"].eq("Checked In")]
    .groupby("Member_ID")["Date"]
    .min()
    .rename("First_Checkin")
)
totals = (
    attendance[attendance["Status"].eq("Checked In")]
    .groupby("Member_ID")
    .size()
    .rename("Total_Checkins")
)
features = features.set_index("Member_ID").join(first_seen).join(totals).reset_index()
features["Months_Active"] = (
    (
        features["Last_Checkin"].dt.to_period("M") - features["First_Checkin"].dt.to_period("M")
    ).apply(lambda p: p.n)
    + 1
).clip(lower=1)
features["Visits_per_Month"] = (features["Total_Checkins"] / features["Months_Active"]).fillna(0)

# Retail spend
spend = sales.groupby("Member_ID")["Amount_USD"].sum().rename("Retail_Spend_USD")
features = features.set_index("Member_ID").join(spend).reset_index().fillna({"Retail_Spend_USD": 0})

# Label churned
features["Churned"] = features["Member_ID"].isin(set(cancellations["Member_ID"])).astype(int)

display(features.head(10))
print(
    "Heuristic: Members with high 'Days_Since_Last' and low 'Visits_per_Month' are at higher risk."
)